# Module 3.1: Harris Corner Detector

## 學習目標

本 notebook 將帶你實作 **Harris Corner Detector**：

1. 理解角點 (Corner) 的定義
2. 結構張量 (Structure Tensor) 的概念
3. Harris Corner Response Function
4. 從零實作完整的 Harris 角點偵測

## 前置要求
- 完成 Module 1（卷積、Sobel）
- 完成 Module 0（矩陣運算）

## 參考資料
- Harris & Stephens, 1988

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
from PIL import Image
import sys
sys.path.append('../../')

print("環境設定完成！")

## 複習：基礎函數

In [ ]:
# 卷積函數
def conv2d(image, kernel, padding='same'):
    """2D 卷積"""
    kH, kW = kernel.shape
    
    if padding == 'valid':
        pad_h, pad_w = 0, 0
    elif padding == 'same':
        pad_h = (kH - 1) // 2
        pad_w = (kW - 1) // 2
    else:
        pad_h = pad_w = padding
    
    if pad_h > 0 or pad_w > 0:
        image = np.pad(image, ((pad_h, pad_h), (pad_w, pad_w)), mode='constant')
    
    H, W = image.shape
    out_H = H - kH + 1
    out_W = W - kW + 1
    
    output = np.zeros((out_H, out_W), dtype=np.float64)
    
    for i in range(out_H):
        for j in range(out_W):
            output[i, j] = np.sum(image[i:i+kH, j:j+kW] * kernel)
    
    return output

# Gaussian kernel
def gaussian_kernel(size, sigma):
    """建立 Gaussian kernel"""
    half = size // 2
    x = np.arange(-half, half + 1)
    X, Y = np.meshgrid(x, x)
    gaussian = np.exp(-(X**2 + Y**2) / (2 * sigma**2))
    return gaussian / gaussian.sum()

def gaussian_blur(image, sigma):
    """Gaussian 平滑"""
    size = int(np.ceil(6 * sigma)) | 1
    kernel = gaussian_kernel(size, sigma)
    return conv2d(image, kernel, padding='same')

# Sobel kernels
SOBEL_X = np.array([[-1, 0, 1], [-2, 0, 2], [-1, 0, 1]], dtype=np.float64)
SOBEL_Y = np.array([[-1, -2, -1], [0, 0, 0], [1, 2, 1]], dtype=np.float64)

print("基礎函數已載入！")

## 建立測試影像

In [ ]:
def create_test_image(size=200):
    """
    建立一個包含各種特徵（角點、邊緣、平坦區域）的測試影像。
    """
    np.random.seed(42)
    image = np.ones((size, size), dtype=np.float64) * 80
    
    # 多個方塊（有明顯的角點）
    image[30:70, 30:70] = 200    # 大方塊
    image[30:60, 100:130] = 180  # 中方塊
    image[120:160, 40:100] = 160 # 長方形
    image[100:140, 130:170] = 190 # 另一個方塊
    
    # 三角形
    for i in range(40):
        x_start = 60 - i//2
        x_end = 60 + i//2
        if 150 + i < size:
            image[150+i, max(0, x_start):min(size, x_end)] = 170
    
    # 加入輕微雜訊
    noise = np.random.randn(size, size) * 3
    image = image + noise
    
    return np.clip(image, 0, 255)

# 建立並顯示
test_image = create_test_image(200)

plt.figure(figsize=(8, 8))
plt.imshow(test_image, cmap='gray', vmin=0, vmax=255)
plt.title('Test Image (with corners)')
plt.colorbar()
plt.show()

---

# Part 1: 角點的定義

## 直覺理解

考慮一個小視窗在影像上移動：

| 區域類型 | 移動效果 |
|----------|----------|
| **Flat (平坦)** | 任何方向移動，灰階都不變 |
| **Edge (邊緣)** | 沿邊緣移動不變，垂直於邊緣移動才變 |
| **Corner (角點)** | 往任何方向移動，灰階都會變化 |

## 數學表示

設 $I(x, y)$ 是影像，考慮視窗內的像素移動 $(u, v)$：

$$
E(u, v) = \sum_{x, y} w(x, y) [I(x+u, y+v) - I(x, y)]^2
$$

其中 $w(x, y)$ 是視窗權重（通常是 Gaussian）。

### 泰勒展開近似

對 $I(x+u, y+v)$ 做一階泰勒展開：

$$
I(x+u, y+v) \approx I(x, y) + I_x u + I_y v
$$

代入 $E(u, v)$：

$$
E(u, v) \approx \sum_{x, y} w(x, y) [I_x u + I_y v]^2
$$

展開並整理成矩陣形式：

$$
E(u, v) \approx \begin{bmatrix} u & v \end{bmatrix}
M
\begin{bmatrix} u \\ v \end{bmatrix}
$$

In [ ]:
# 視覺化 flat、edge、corner 的差異

fig, axes = plt.subplots(2, 3, figsize=(15, 10))

# 建立三種類型的小區域
flat_region = np.ones((50, 50)) * 100

edge_region = np.ones((50, 50)) * 50
edge_region[:, 25:] = 200

corner_region = np.ones((50, 50)) * 50
corner_region[25:, 25:] = 200

regions = [flat_region, edge_region, corner_region]
titles = ['Flat', 'Edge', 'Corner']

for i, (region, title) in enumerate(zip(regions, titles)):
    # 顯示區域
    axes[0, i].imshow(region, cmap='gray', vmin=0, vmax=255)
    axes[0, i].set_title(f'{title} Region')
    axes[0, i].axis('off')
    
    # 計算梯度
    Ix = conv2d(region, SOBEL_X, padding='same')
    Iy = conv2d(region, SOBEL_Y, padding='same')
    
    # 繪製梯度方向
    y, x = np.mgrid[0:50:5, 0:50:5]
    u = Ix[::5, ::5]
    v = Iy[::5, ::5]
    
    axes[1, i].imshow(region, cmap='gray', vmin=0, vmax=255, alpha=0.5)
    axes[1, i].quiver(x, y, u, v, color='red', scale=1000)
    axes[1, i].set_title(f'{title} Gradients')
    axes[1, i].axis('off')

plt.tight_layout()
plt.show()

print("觀察：")
print("- Flat：幾乎沒有梯度")
print("- Edge：梯度只在一個方向（垂直於邊緣）")
print("- Corner：梯度在多個方向")

---

# Part 2: Structure Tensor（結構張量）

## 定義

**Structure Tensor（結構張量）** $M$ 定義為：

$$
M = \begin{bmatrix}
\sum_w I_x^2 & \sum_w I_x I_y \\
\sum_w I_x I_y & \sum_w I_y^2
\end{bmatrix}
= \begin{bmatrix}
A & C \\
C & B
\end{bmatrix}
$$

其中 $\sum_w$ 表示用 Gaussian 加權的視窗內求和。

## 特徵值分析

$M$ 的兩個特徵值 $\lambda_1, \lambda_2$ 決定了區域類型：

| $\lambda_1$ | $\lambda_2$ | 區域類型 |
|-------------|-------------|----------|
| 小 | 小 | Flat |
| 大 | 小 | Edge |
| 大 | 大 | Corner |

In [ ]:
def compute_structure_tensor(image, sigma=1.0):
    """
    計算結構張量的各個分量。
    
    Parameters
    ----------
    image : np.ndarray
        輸入灰階影像
    sigma : float
        Gaussian 平滑的 sigma（用於計算梯度和加權）
    
    Returns
    -------
    A : np.ndarray
        Σ Ix²
    B : np.ndarray
        Σ Iy²
    C : np.ndarray
        Σ IxIy
    
    實作步驟：
    1. 計算 Ix, Iy（Sobel）
    2. 計算 Ix², Iy², IxIy
    3. 用 Gaussian 平滑上述三個（相當於加權求和）
    """
    # Step 1: 計算梯度
    Ix = conv2d(image, SOBEL_X, padding='same')
    Iy = conv2d(image, SOBEL_Y, padding='same')
    
    # Step 2: 計算二次項
    Ix2 = Ix * Ix
    Iy2 = Iy * Iy
    IxIy = Ix * Iy
    
    # Step 3: Gaussian 平滑（加權求和）
    A = gaussian_blur(Ix2, sigma)   # Σ Ix²
    B = gaussian_blur(Iy2, sigma)   # Σ Iy²
    C = gaussian_blur(IxIy, sigma)  # Σ IxIy
    
    return A, B, C

# 測試
A, B, C = compute_structure_tensor(test_image, sigma=2.0)

fig, axes = plt.subplots(1, 4, figsize=(16, 4))

axes[0].imshow(test_image, cmap='gray', vmin=0, vmax=255)
axes[0].set_title('Original')

im1 = axes[1].imshow(A, cmap='hot')
axes[1].set_title('A = Σ Ix²')
plt.colorbar(im1, ax=axes[1])

im2 = axes[2].imshow(B, cmap='hot')
axes[2].set_title('B = Σ Iy²')
plt.colorbar(im2, ax=axes[2])

im3 = axes[3].imshow(np.abs(C), cmap='hot')
axes[3].set_title('|C| = |Σ IxIy|')
plt.colorbar(im3, ax=axes[3])

for ax in axes:
    ax.axis('off')

plt.tight_layout()
plt.show()

print("觀察：")
print("- A (Ix²) 在垂直邊緣處較大")
print("- B (Iy²) 在水平邊緣處較大")
print("- C (IxIy) 在角點處較大")

---

# Part 3: Harris Corner Response

## 為什麼不直接算特徵值？

- 計算特徵值需要解二次方程，計算量大
- Harris 提出一個更簡單的函數

## Harris Response Function

$$
R = \det(M) - k \cdot \text{trace}(M)^2
$$

其中：
- $\det(M) = AB - C^2 = \lambda_1 \lambda_2$
- $\text{trace}(M) = A + B = \lambda_1 + \lambda_2$
- $k$ 是經驗參數，通常取 0.04 ~ 0.06

## R 值的解釋

| R 值 | 解釋 |
|------|------|
| R >> 0 | Corner（角點）|
| R ≈ 0 | Flat（平坦）|
| R << 0 | Edge（邊緣）|

In [ ]:
def compute_harris_response(A, B, C, k=0.05):
    """
    計算 Harris corner response。
    
    Parameters
    ----------
    A, B, C : np.ndarray
        結構張量的分量
    k : float
        Harris 參數（通常 0.04-0.06）
    
    Returns
    -------
    R : np.ndarray
        Harris response
    
    公式：R = det(M) - k * trace(M)²
        = (A*B - C²) - k * (A + B)²
    """
    # det(M) = A*B - C*C
    det = A * B - C * C
    
    # trace(M) = A + B
    trace = A + B
    
    # Harris response
    R = det - k * trace * trace
    
    return R

# 計算 Harris response
R = compute_harris_response(A, B, C, k=0.05)

fig, axes = plt.subplots(1, 3, figsize=(15, 5))

axes[0].imshow(test_image, cmap='gray', vmin=0, vmax=255)
axes[0].set_title('Original')
axes[0].axis('off')

# 顯示 R（正值 = corner，負值 = edge）
im1 = axes[1].imshow(R, cmap='RdBu', vmin=-np.abs(R).max(), vmax=np.abs(R).max())
axes[1].set_title('Harris Response R')
axes[1].axis('off')
plt.colorbar(im1, ax=axes[1])

# 只顯示正值（角點候選）
R_positive = np.maximum(R, 0)
im2 = axes[2].imshow(R_positive, cmap='hot')
axes[2].set_title('R > 0 (Corner candidates)')
axes[2].axis('off')
plt.colorbar(im2, ax=axes[2])

plt.tight_layout()
plt.show()

print("觀察：")
print("- 紅色區域（R > 0）：角點候選")
print("- 藍色區域（R < 0）：邊緣")
print("- 白色區域（R ≈ 0）：平坦")

---

# Part 4: Non-Maximum Suppression

## 目的

找出 R 的**局部最大值**，避免在同一個角點附近偵測到多個點。

## 方法

1. 對每個像素，檢查它是否是鄰域內的最大值
2. 只保留局部最大值

In [ ]:
def non_maximum_suppression(R, window_size=5):
    """
    對 Harris response 進行非極大值抑制。
    
    Parameters
    ----------
    R : np.ndarray
        Harris response
    window_size : int
        鄰域大小（必須是奇數）
    
    Returns
    -------
    nms : np.ndarray
        NMS 後的 response（只有局部最大值非零）
    """
    H, W = R.shape
    half = window_size // 2
    nms = np.zeros_like(R)
    
    for i in range(half, H - half):
        for j in range(half, W - half):
            # 取出鄰域
            window = R[i-half:i+half+1, j-half:j+half+1]
            
            # 如果當前像素是鄰域內的最大值，保留
            if R[i, j] == window.max() and R[i, j] > 0:
                nms[i, j] = R[i, j]
    
    return nms

# 進行 NMS
R_nms = non_maximum_suppression(R, window_size=7)

fig, axes = plt.subplots(1, 2, figsize=(12, 5))

axes[0].imshow(R_positive, cmap='hot')
axes[0].set_title('Before NMS')

axes[1].imshow(R_nms, cmap='hot')
axes[1].set_title('After NMS')

for ax in axes:
    ax.axis('off')

plt.tight_layout()
plt.show()

print(f"NMS 前的非零像素數：{np.sum(R_positive > 0)}")
print(f"NMS 後的非零像素數：{np.sum(R_nms > 0)}")

---

# Part 5: 完整的 Harris Corner Detector

In [ ]:
def harris_corner_detector(image, sigma=2.0, k=0.05, threshold=0.01, nms_size=7):
    """
    完整的 Harris 角點偵測。
    
    Parameters
    ----------
    image : np.ndarray
        灰階影像
    sigma : float
        Gaussian 平滑的 sigma
    k : float
        Harris 參數
    threshold : float
        閾值（相對於最大 response 的比例）
    nms_size : int
        NMS 視窗大小
    
    Returns
    -------
    corners : list of (y, x)
        角點座標列表
    response : np.ndarray
        Harris response map
    
    流程：
    1. 計算結構張量
    2. 計算 Harris response
    3. Non-maximum suppression
    4. 閾值過濾
    """
    # Step 1: 計算結構張量
    A, B, C = compute_structure_tensor(image, sigma)
    
    # Step 2: 計算 Harris response
    R = compute_harris_response(A, B, C, k)
    
    # Step 3: Non-maximum suppression
    R_nms = non_maximum_suppression(R, nms_size)
    
    # Step 4: 閾值過濾
    thresh_value = threshold * R_nms.max()
    corners_y, corners_x = np.where(R_nms > thresh_value)
    
    # 組成 (y, x) 列表
    corners = list(zip(corners_y, corners_x))
    
    return corners, R

# 測試
corners, response = harris_corner_detector(test_image, sigma=2.0, k=0.05, threshold=0.01)

print(f"偵測到 {len(corners)} 個角點")

# 視覺化
fig, axes = plt.subplots(1, 2, figsize=(14, 6))

axes[0].imshow(response, cmap='hot')
axes[0].set_title('Harris Response')
axes[0].axis('off')

# 在原圖上標記角點
axes[1].imshow(test_image, cmap='gray', vmin=0, vmax=255)
for y, x in corners:
    axes[1].scatter(x, y, c='red', s=50, marker='+')
axes[1].set_title(f'Detected Corners ({len(corners)} points)')
axes[1].axis('off')

plt.tight_layout()
plt.show()

In [ ]:
# 測試不同參數
fig, axes = plt.subplots(2, 3, figsize=(15, 10))

# 不同的 threshold
thresholds = [0.001, 0.01, 0.05]
for ax, thresh in zip(axes[0], thresholds):
    corners, _ = harris_corner_detector(test_image, threshold=thresh)
    ax.imshow(test_image, cmap='gray', vmin=0, vmax=255)
    for y, x in corners:
        ax.scatter(x, y, c='red', s=30, marker='+')
    ax.set_title(f'threshold={thresh}, {len(corners)} corners')
    ax.axis('off')

# 不同的 sigma
sigmas = [1.0, 2.0, 4.0]
for ax, sigma in zip(axes[1], sigmas):
    corners, _ = harris_corner_detector(test_image, sigma=sigma)
    ax.imshow(test_image, cmap='gray', vmin=0, vmax=255)
    for y, x in corners:
        ax.scatter(x, y, c='red', s=30, marker='+')
    ax.set_title(f'sigma={sigma}, {len(corners)} corners')
    ax.axis('off')

plt.suptitle('Effect of Parameters')
plt.tight_layout()
plt.show()

---

# Part 6: 練習題

## 練習 1: 視覺化特徵值橢圓

In [ ]:
def compute_eigenvalues(A, B, C):
    """
    計算 structure tensor 的特徵值。
    
    對於 2x2 矩陣 [[A, C], [C, B]]，特徵值是：
    λ = (A + B)/2 ± sqrt((A - B)²/4 + C²)
    
    Returns
    -------
    lambda1, lambda2 : np.ndarray
        兩個特徵值（λ1 >= λ2）
    """
    # 中間值
    tmp = np.sqrt(((A - B) / 2)**2 + C**2)
    
    lambda1 = (A + B) / 2 + tmp  # 較大的特徵值
    lambda2 = (A + B) / 2 - tmp  # 較小的特徵值
    
    return lambda1, lambda2

# 計算特徵值
lambda1, lambda2 = compute_eigenvalues(A, B, C)

# 視覺化
fig, axes = plt.subplots(1, 3, figsize=(15, 5))

im0 = axes[0].imshow(lambda1, cmap='hot')
axes[0].set_title('λ₁ (larger eigenvalue)')
plt.colorbar(im0, ax=axes[0])

im1 = axes[1].imshow(lambda2, cmap='hot')
axes[1].set_title('λ₂ (smaller eigenvalue)')
plt.colorbar(im1, ax=axes[1])

# λ₁ vs λ₂ 的關係可以區分 corner/edge/flat
ratio = np.minimum(lambda1 / (lambda2 + 1e-10), 100)
im2 = axes[2].imshow(ratio, cmap='hot', vmax=10)
axes[2].set_title('λ₁ / λ₂ (ratio)')
plt.colorbar(im2, ax=axes[2])

for ax in axes:
    ax.axis('off')

plt.tight_layout()
plt.show()

print("觀察：")
print("- Corner：λ₁ 和 λ₂ 都大，ratio ≈ 1")
print("- Edge：λ₁ 大，λ₂ 小，ratio 大")
print("- Flat：λ₁ 和 λ₂ 都小")

## 練習 2: Shi-Tomasi 角點偵測

In [ ]:
def shi_tomasi_corner_detector(image, sigma=2.0, threshold=0.01, nms_size=7):
    """
    Shi-Tomasi 角點偵測（又稱 Good Features to Track）。
    
    與 Harris 的差異：
    - Harris: R = λ₁λ₂ - k(λ₁ + λ₂)²
    - Shi-Tomasi: R = min(λ₁, λ₂)
    
    Shi-Tomasi 更穩定，因為直接使用較小的特徵值。
    """
    # 計算結構張量
    A, B, C = compute_structure_tensor(image, sigma)
    
    # 計算特徵值
    lambda1, lambda2 = compute_eigenvalues(A, B, C)
    
    # Shi-Tomasi response: min(λ₁, λ₂)
    R = np.minimum(lambda1, lambda2)
    
    # NMS
    R_nms = non_maximum_suppression(R, nms_size)
    
    # 閾值
    thresh_value = threshold * R_nms.max()
    corners_y, corners_x = np.where(R_nms > thresh_value)
    
    corners = list(zip(corners_y, corners_x))
    
    return corners, R

# 比較 Harris 和 Shi-Tomasi
harris_corners, harris_R = harris_corner_detector(test_image)
shi_tomasi_corners, shi_tomasi_R = shi_tomasi_corner_detector(test_image)

fig, axes = plt.subplots(1, 2, figsize=(14, 6))

axes[0].imshow(test_image, cmap='gray', vmin=0, vmax=255)
for y, x in harris_corners:
    axes[0].scatter(x, y, c='red', s=50, marker='+')
axes[0].set_title(f'Harris ({len(harris_corners)} corners)')
axes[0].axis('off')

axes[1].imshow(test_image, cmap='gray', vmin=0, vmax=255)
for y, x in shi_tomasi_corners:
    axes[1].scatter(x, y, c='blue', s=50, marker='+')
axes[1].set_title(f'Shi-Tomasi ({len(shi_tomasi_corners)} corners)')
axes[1].axis('off')

plt.tight_layout()
plt.show()

## 練習 3: 完整的角點偵測類別

In [ ]:
class CornerDetector:
    """
    角點偵測工具類別。
    """
    
    @staticmethod
    def compute_structure_tensor(image, sigma=2.0):
        """計算結構張量"""
        Ix = conv2d(image, SOBEL_X, padding='same')
        Iy = conv2d(image, SOBEL_Y, padding='same')
        
        A = gaussian_blur(Ix * Ix, sigma)
        B = gaussian_blur(Iy * Iy, sigma)
        C = gaussian_blur(Ix * Iy, sigma)
        
        return A, B, C
    
    @classmethod
    def harris(cls, image, sigma=2.0, k=0.05, threshold=0.01, nms_size=7):
        """Harris 角點偵測"""
        A, B, C = cls.compute_structure_tensor(image, sigma)
        
        det = A * B - C * C
        trace = A + B
        R = det - k * trace * trace
        
        R_nms = non_maximum_suppression(R, nms_size)
        thresh_value = threshold * R_nms.max()
        
        corners_y, corners_x = np.where(R_nms > thresh_value)
        return list(zip(corners_y, corners_x)), R
    
    @classmethod
    def shi_tomasi(cls, image, sigma=2.0, threshold=0.01, nms_size=7):
        """Shi-Tomasi 角點偵測"""
        A, B, C = cls.compute_structure_tensor(image, sigma)
        
        tmp = np.sqrt(((A - B) / 2)**2 + C**2)
        lambda2 = (A + B) / 2 - tmp  # 較小的特徵值
        
        R = lambda2
        R_nms = non_maximum_suppression(R, nms_size)
        thresh_value = threshold * R_nms.max()
        
        corners_y, corners_x = np.where(R_nms > thresh_value)
        return list(zip(corners_y, corners_x)), R
    
    @staticmethod
    def draw_corners(image, corners, color='red', marker='+', size=50):
        """在影像上繪製角點"""
        plt.figure(figsize=(8, 8))
        plt.imshow(image, cmap='gray', vmin=0, vmax=255)
        for y, x in corners:
            plt.scatter(x, y, c=color, s=size, marker=marker)
        plt.title(f'{len(corners)} corners detected')
        plt.axis('off')
        plt.show()

# 測試
cd = CornerDetector

corners, _ = cd.harris(test_image)
cd.draw_corners(test_image, corners)

---

# 總結

## Harris Corner Detector 核心概念

| 步驟 | 目的 | 方法 |
|------|------|------|
| 1. 計算梯度 | 找影像變化 | Sobel |
| 2. Structure Tensor | 描述局部梯度分布 | Ix², Iy², IxIy |
| 3. Harris Response | 區分 corner/edge/flat | det - k*trace² |
| 4. NMS | 找局部最大值 | 鄰域比較 |

## 關鍵公式

- Structure Tensor: $M = \begin{bmatrix} \Sigma I_x^2 & \Sigma I_x I_y \\ \Sigma I_x I_y & \Sigma I_y^2 \end{bmatrix}$

- Harris Response: $R = \det(M) - k \cdot \text{trace}(M)^2$

## 下一步

在 02_sift.ipynb 中，我們將學習 **SIFT** 的尺度空間概念和 Gaussian Pyramid。

In [ ]:
# 最終驗證
print("=== 函數驗證 ===")

test_funcs = [
    ("conv2d(image, kernel)", lambda: conv2d(test_image, SOBEL_X)),
    ("gaussian_blur(image, sigma)", lambda: gaussian_blur(test_image, 2.0)),
    ("compute_structure_tensor(image)", lambda: compute_structure_tensor(test_image)),
    ("compute_harris_response(A, B, C)", lambda: compute_harris_response(A, B, C)),
    ("non_maximum_suppression(R)", lambda: non_maximum_suppression(R)),
    ("harris_corner_detector(image)", lambda: harris_corner_detector(test_image)),
    ("compute_eigenvalues(A, B, C)", lambda: compute_eigenvalues(A, B, C)),
    ("shi_tomasi_corner_detector(image)", lambda: shi_tomasi_corner_detector(test_image)),
    ("CornerDetector.harris(image)", lambda: CornerDetector.harris(test_image)),
    ("CornerDetector.shi_tomasi(image)", lambda: CornerDetector.shi_tomasi(test_image)),
]

for name, func in test_funcs:
    try:
        result = func()
        print(f"✓ {name}")
    except Exception as e:
        print(f"✗ {name}: {e}")

print("\n所有函數驗證完成！")